In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
import requests
from torchvision import transforms

In [6]:
import os 
import torch
from PIL import Image
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

# 区分清楚：class 子类(父类)，def 函数(参数) ，这里是继承pytorch中的父类 Dataset 并进行改写
class CelebADataset(Dataset):
    # __init__是类的默认构造函数，self必写，root 和 img_shape 是子类的默认必有的参数，之后传参的时候要用的；
    def __init__(self, root, img_shape=(64, 64)) -> None:
        super().__init__()
        self.root = root
        self.img_shape = img_shape
        # 获取根目录 root 下的所有 directory
        self.filenames = sorted(os.listdir(root))

    # 返回文件名的数量，即数据集的大小
    def __len__(self) -> int:
        return len(self.filenames)

    def __getitem__(self, index: int):
        # 使用 os.path.join 将根目录和其中获取的 directory name 组合成完整的文件路径。
        path = os.path.join(self.root, self.filenames[index])
        img = Image.open(path).convert('RGB')
        # 图片处理流水线，裁剪、放缩、torch张量
        pipeline = transforms.Compose([
            transforms.CenterCrop(168),
            transforms.Resize(self.img_shape),
            transforms.ToTensor()
        ])
        return pipeline(img)
    
# DataLoader 批量加载数据，设置 epoch 大小为 16，并随机打乱数据（shuffle=True）
# 如果你想表示一个目录，通常在路径末尾加上斜杠是一个好习惯; 如果想表示文件，路径末尾不应加斜杠：    
def get_dataloader(root="/data/home/huangyx/workspace/diffusion_learn/VAE/img_align_celeba/", **kwargs):
    # root 和 img_shape 是定义的默认参数，一定要有的；img_shape已经固定了这里可以不传，root 是一定要传的    
    # **kwargs 说明允许传入其他参数
    # crop, resize 等操作已经在定义类时一同定义了；所以传参传入后，子类定义的过程就会进行一遍；这里输出的 dataset 也是经过子类加工的对象
    dataset = CelebADataset(root, **kwargs)
    return DataLoader(dataset, 16, shuffle=True)


# 为了验证Dataloader的正确性，我们可以写一些脚本来查看Dataloader里的一个batch的图片。(和主任务没关系)
if __name__ == '__main__':
    dataloader = get_dataloader()
    img = next(iter(dataloader))
    print(img.shape)
    # Concat 4x4 images
    N, C, H, W = img.shape
    assert N == 16
    img = torch.permute(img, (1, 0, 2, 3))
    img = torch.reshape(img, (C, N // 4, 4 * H, W))
    img = torch.permute(img, (0, 2, 1, 3))
    img = torch.reshape(img, (C, 4 * H, 4 * W))
    img = transforms.ToPILImage()(img)
    img.save('/data/home/huangyx/workspace/diffusion_learn/VAE/tmp_test.jpg')

torch.Size([16, 3, 64, 64])
